In [23]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

In [24]:
# Freeze base model
# base_model.trainable = FIXME
base_model.trainable = False

In [25]:
# # Create inputs with correct shape
# inputs = keras.Input(shape=(224, 224, 3))

# x = base_model(inputs, training=False)
# x = keras.layers.GlobalAveragePooling2D()(x)

# # Add final dense layer
# outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# # Combine inputs and outputs to create model
# model = keras.Model(inputs, outputs)

inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to False 
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)



In [26]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


In [27]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# datagen_train = ImageDataGenerator(FIXME)
# datagen_valid = ImageDataGenerator(FIXME)

# Create a data generator
datagen_train = ImageDataGenerator(
    samplewise_center=True,  # set each sample mean to 0
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
)  # we don't expect Bo to be upside-down so we will not flip vertically

# No need to augment validation data
datagen_valid = ImageDataGenerator(samplewise_center=True)

In [29]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    "data/fruits/train",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)
# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    "data/fruits/valid",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20
37/36 [==============================] - 17s 454ms/step - loss: 6.9435 - accuracy: 0.1032 - val_loss: 8.4661 - val_accuracy: 0.0881
Epoch 2/20
37/36 [==============================] - 17s 454ms/step - loss: 7.3729 - accuracy: 0.1041 - val_loss: 7.2159 - val_accuracy: 0.2188
Epoch 3/20
37/36 [==============================] - 17s 458ms/step - loss: 6.2546 - accuracy: 0.1946 - val_loss: 5.9410 - val_accuracy: 0.2310
Epoch 4/20
37/36 [==============================] - 17s 452ms/step - loss: 6.6924 - accuracy: 0.2437 - val_loss: 6.8991 - val_accuracy: 0.2918
Epoch 5/20
37/36 [==============================] - 17s 451ms/step - loss: 7.5371 - accuracy: 0.2758 - val_loss: 6.6105 - val_accuracy: 0.2736
Epoch 6/20
37/36 [==============================] - 17s 454ms/step - loss: 7.8607 - accuracy: 0.2809 - val_loss: 7.1468 - val_accuracy: 0.2553
Epoch 7/20
33/36 [=========================>....] - ETA: 1s - loss: 7.8071 - accuracy: 0.2585

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
# model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
#               loss = FIXME , metrics = FIXME)

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10)

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

In [ ]:
from run_assessment import run_assessment

In [ ]:
run_assessment(model, valid_it)